In [7]:
from glob import glob
import os
routes = glob('/data/ephemeral/home/datas/text/*')
data_route = 'datas/test.csv'

In [2]:
import re
from tqdm import tqdm
def parse_wiki_file(file_path):
    documents = []
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # 문서별로 분리 (각 문서는 <doc>로 시작하고 </doc>로 끝남)
    doc_pattern = re.compile(r'<doc id="(.*?)" url="(.*?)" title="(.*?)">(.*?)</doc>', re.DOTALL)
    matches = doc_pattern.findall(content)
    
    for match in matches:
        doc_id, url, title, text = match
        documents.append({
            "id": doc_id,
            "url": url,
            "title": title,
            "text": text.strip()
        })
    
    return documents
result = []
for route in routes:
    for r in tqdm(glob(route + '/*'), desc = 'collecting...'):
        result.extend(parse_wiki_file(r))


collecting...: 100%|██████████| 100/100 [00:01<00:00, 59.71it/s]


In [8]:
import pandas as pd

df = pd.DataFrame(result)
df['doc_len'] = df['text'].apply(lambda x: len(x))

In [9]:
df = df[(df['doc_len'] > 100) & (df['doc_len'] < 2000)]

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def text_split(doc):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    chunks = text_splitter.split_text(doc)
    return chunks

df['splitted_text'] = df['text'].apply(text_split)

In [11]:
df['splitted_text'].sample(3)

205343    [북극 세계기록보관소, 북극 세계기록보관소(Arctic World Archive, ...
827610    [청주 문의문산관, 청주 문의문산관(淸州 文義文山館)은 충청북도 청주시 상당구 문의...
775180    [이지안 (미스코리아)\n\n이지안(본명: 이은희, 1977년 7월 24일~)는 대...
Name: splitted_text, dtype: object

In [15]:
data_route.split('.csv')[0]+'+' +'keyword.csv'

'datas/test+keyword.csv'

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from kiwipiepy import Kiwi
from keybert import KeyBERT
from transformers import pipeline
from ast import literal_eval
from tqdm import tqdm

kw_model = KeyBERT(model='sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

# 텍스트에서 주요 키워드 추출 함수 (KeyBERT 사용)
def extract_keywords(doc, top_n=5, ngram = 2):
    kiwi = Kiwi()
    # 문장을 형태소 분석하여 명사, 형용사, 동사 등을 추출
    tokens = kiwi.tokenize(doc)
    words = [token.form for token in tokens if token.tag.startswith(('NN', 'VV', 'VA'))]  # 명사(NN), 동사(VV), 형용사(VA)만 추출

    # KeyBERT 모델을 사용하여 주요 키워드 추출
    keywords = kw_model.extract_keywords(' '.join(words), keyphrase_ngram_range=(1, ngram), stop_words=None, top_n=top_n)
    
    # 결과에서 키워드만 추출하여 리스트로 반환
    return [keyword[0] for keyword in keywords]

data = pd.read_csv(data_route)
data['dic'] = data['problems'].apply(lambda x: literal_eval(x))
data['question'] = data['dic'].apply(lambda x: x['question'])


tqdm.pandas()

# data['keywords'] = data.progress_apply(lambda x: extract_keywords(x['paragraph'] + ' ' + x['question']), axis=1)
data['keywords_1'] = data.progress_apply(lambda x: extract_keywords(x['paragraph'] + ' ' + x['question'], ngram = 1), axis=1)
# 위 코드는 ngram 2개까지 아래코드는 ngram 1개만

file_name = data_route.split('.csv')[0]+'+' +'keyword.csv'
data[['id', 'paragraph', 'problems', 'question_plus', 'keywords_1']].to_csv(file_name, index = False)
data = pd.read_csv(file_name)

docs = []
titles_set = set(df['title'].values)  
cnt = 0

for words in tqdm(data['keywords_1'].values):
    for word in words:
        if word in titles_set:
            docs.append(df[df['title'] == word].to_dict('records'))

# 위키 문서에서 title이 매칭되는 애들만 뽑기

flattened_docs = [item for sublist in docs for item in sublist]

df = pd.DataFrame(flattened_docs)

100%|██████████| 869/869 [02:20<00:00,  6.18it/s]


In [23]:
corpus = []
for i ,item in tqdm(df.iterrows()):
    corpus += item['splitted_text']
corpus = list(filter(lambda x: len(x) > 100, corpus))
print(len(corpus))


2119it [00:00, 22013.55it/s]

4658


In [24]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
import os
import gc
import torch
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.schema import Document
gc.collect()
torch.cuda.empty_cache()
model_name = "intfloat/multilingual-e5-large-instruct"
ko_embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cuda"},
    encode_kwargs={'normalize_embeddings': True}
)

docs = [Document(page_content=text, metadata={"source": f"doc_{i}"}) for i, text in enumerate(corpus)]

print('making bm25..')
# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = 2
print('making dense retrieval..')
faiss_vectorstore = FAISS.from_documents(docs, ko_embedding)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})
ensemble_retriever = EnsembleRetriever(
retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)   



/tmp/ipykernel_155974/3318276372.py:15: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  ko_embedding = HuggingFaceEmbeddings(


making bm25..
making dense retrieval..


In [26]:
from ast import literal_eval
data = pd.read_csv('datas/test.csv')
data['dic'] = data['problems'].apply(lambda x: literal_eval(x))
data['question'] = data['dic'].apply(lambda x: x['question'])

In [27]:
idx = 42
result = ensemble_retriever.get_relevant_documents(data['paragraph'][idx] + data['question'][idx])
print(data['paragraph'][idx])
print(data['question'][idx])
print(result)

/tmp/ipykernel_155974/1343393623.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = ensemble_retriever.get_relevant_documents(data['paragraph'][idx] + data['question'][idx])


동물 병원 원장 A( 3 5세 )는 차량의 소유자로부터 공작물인   차량을 장기간 임차하여 사용하고 있다 .  A는 고객 갑이 치료를  위해 입원시킨 반려견의 재활 운동을 위해 그 차량을 운전하여  직원 B( 2 1세 )와 함께 공원으로 갔다 .  A는 차량을 주차하고 B 와  함께 반려견을 데리고 산책하던 중 B가 부주의로 반려견의 발을  밟아 상처를 입혔다 .  놀란 반려견이 지나가던 을의 다리를 물어  2주간의 치료를 요하는 상처를 입혔고,  을은 이로 인해 정신적 으로도 큰 충격을 받았다 .  그 사이 A가 주차해 두었던 차량에서  불이 났고,  이로 인해 옆에 주차되어 있던 병 소유 차량이 파손 되는 재산상 손해가 발생하였다 .  사고 조사 결과,  화재의 원인은  차량에 대한 비전문가인 소유자가 해당 차량을 직접 수리하여   발생한 보존상의 하자에 의한 것으로 밝혀졌다 .
다음 사례에 대한 법적 판단으로 옳은 것은 ?
[Document(metadata={'source': 'doc_2995'}, page_content='인간이 발명한 최초의 스포츠 장비는 공이었다. 고대 이집트에서는 돌 던지기가 아이들이 가장 좋아하는 놀이였지만, 잘못 던진 돌은 아이에게 상처를 입힐 수 있었다. 그래서 이집트인들은 던지기에 덜 위험한 것을 찾고 있었다. 그리고 그들은 아마도 최초의 공이었던 것을 개발했다. 그것들은 처음에 실로 함께 묶은 풀이나 나뭇잎으로 만들어졌고, 나중에는 바느질해 붙이고 속에 깃털이나 짚으로 채워 넣은 동물 가죽 조각으로 만들어졌다. 시민들이 스포츠에 직접 참가하는 일이 드물었던 고대 로마에서도 공놀이는 가장 인기 높은 스포츠였다.'), Document(metadata={'source': 'doc_1456'}, page_content='학설.\n여러 견해가 있으며 어느 하나의 견해에만 입각하여 죄수를 결정할 수 없다. 죄란 구성 요건을 전제로 한 개념이므로, 구성 요건 표준설을 우선적 기준으로 삼되 행위의 개수 범죄 의사 및 법익 등을 종합적으로

In [28]:
from tqdm import tqdm

def generate_hint(row):
    query = row['question'] + ' ' + row['question']
    relevant_docs = ensemble_retriever.get_relevant_documents(query)
    
    # `hint` 문자열 생성
    hint = ''
    for i, relevant_doc in enumerate(relevant_docs):
        hint += f"참고문서{i} : {relevant_doc.page_content}\n"
        if i == 2:
            break  # 3개까지만
    return hint

# tqdm 적용 및 `apply`를 사용하여 새 열 생성
tqdm.pandas()
data['hint'] = train.progress_apply(generate_hint, axis=1)


100%|██████████| 869/869 [00:55<00:00, 15.54it/s]


In [30]:
d = pd.read_csv(data_route)
d['hint'] = data['hint']

In [31]:
print(d['hint'][4])

참고문서0 : 철학에서 다루는 신에 대한 것으로는 “신 없이도 도덕이 가능한가?”라는 것이있다. 일반적으로 신을 믿는 사람은 그 종교에서 요구하는 도덕적 명령을 따르거나, 따르려고 애쓴다. 따라서 예전부터 이러한 신과 종교가 도덕을 가능하게 한다는 주장이 있었다. 이에 대해 다양한 주장이 있으며 “신이 도덕을 명령하기 때문에 도덕을 따라야 하는가, 아니면 그 도덕이 선한 것이기 때문에 따라야 하는가?”와 같은 반박도 있다.
신의 존재에 관한 생각.
리처드 도킨스는 신의 존재 여부에 대해 인간의 생각을 대략적으로 다음과 같이 분류하였다.
출처.
&lt;templatestyles src="각주/styles.css" /&gt;
참고문서1 : 학설.
여러 견해가 있으며 어느 하나의 견해에만 입각하여 죄수를 결정할 수 없다. 죄란 구성 요건을 전제로 한 개념이므로, 구성 요건 표준설을 우선적 기준으로 삼되 행위의 개수 범죄 의사 및 법익 등을 종합적으로 고려하여 각각의 범죄에 합당한 기준을 찾아야 한다.
각주.
&lt;templatestyles src="각주/styles.css" /&gt;



In [32]:
d[['id', 'paragraph', 'problems', 'question_plus', 'hint']].to_csv(file_name)